# Prepare queries

In [2]:
import os
import pickle as p
from p_bloom_filter import build_bloom_filter
from phe import paillier
from p_database import encode_data, encrypt_bloom_filter
from collections import defaultdict, namedtuple

data = None
SEQUENCE_TYPE = "public_addgene_full_sequences"
Gene = namedtuple("Gene", "name, pi, sequence, bloom")

In [3]:
f = 'sample.txt'
with open(f, 'r') as queries_file:
    text = queries_file.read()
    queries = text.splitlines()
queries_file.close()

In [4]:
qs = []
SIZE = 12000
K = 16
H = hash

for query in queries:
    qs.append(build_bloom_filter(query, SIZE, K, H))

In [7]:
public_key, private_key = paillier.generate_paillier_keypair()

In [8]:
enc_qs = []
for query in qs:
    enc_qs.append(encrypt_bloom_filter(query, public_key))

In [10]:
result_set = {}
result_set['public_key'] = public_key
result_set['private_key'] = private_key
result_set['query_list'] = queries
result_set['query_blooms'] = qs
result_set['encrypted_blooms'] = enc_qs

In [11]:
result_set.keys()

dict_keys(['private_key', 'encrypted_blooms', 'query_list', 'public_key', 'query_blooms'])

In [12]:
p.dump(result_set, open('prepared_query_data.p', 'wb'))

# Prepare database

In [13]:
file_loc = os.path.join('../', 'addgene-plasmids-sequences.json')
data = encode_data(file_loc)

Data successfully loaded...


In [14]:
len(data)

5931

In [4]:
p.dump(data, open('prepared_encoded_database.p', 'wb'))